### Метрики в задачах регрессии и классификации
Источники: [Учебник по машинному обучению](https://ml-handbook.ru/); Курс "Машинное обучение", ФКН ВШЭ

Как правило цикл построения и обучения модели выглядит следующим образом:
- Производим обучение модели на **тренировочной выборке**, т.е. используем часть от полного набора данных для обучения алгоритма.
- После этого переходим к этапу **валидации** -- процессу настраивания гиперпараметров модели, подбору архитектуры нейросети и т.д. Затем ищем наилучшие гиперпараметры, проверяя предсказания модели с помощью **метрик качества**. Наконец, фиксируем все выбранные гиперпараметры, которые мы посчитали наилучшими для нашей задачи.
- Используем **тестовые данные**, которые модель никогда не "видела" и проверяем качество на них. 

Не надо тренировать и валидировать модель на тестовых данных! Существует даже отдельный термин -- data leak (утечка данных) -- ситуация, когда тестовые данные попадают в тренировочный датасет.

Поскольку на разных этапах разработки моделей необходимо вычислять метрики качества, то о них стоит поговорить подробнее.

Первое, о чем стоит помнить: *метрика качества* и *функционал качества* (функция потерь) -- это **разные вещи**. *Функция потерь* используется на этапе решения задачи оптимизации. *Метрика* -- внешний критерий качества, зависящий только от предсказаний модели, т.е. она не влияет на процесс обучения модели. Для задачи регрессии, как мы видели, метрика и функция потерь могут совпадать (MSE, MAE), но, например, в задаче классификации метрики сильно отличаются от вида функционала качества.



### Метрики в задаче регрессии
В задаче регрессии ответы принадлежат множеству вещественных чисел, то есть имеем потенциально бесконечное число значений. Примером могут послужить: температура на метеостанции, количество проданных товаров, стоимость квадратного метра квартиры и т.д.
- Mean Squared Error (среднеквадратическая ошибка)
$$MSE = \frac{1}{n}\sum\limits_{i=1}^n\left(y_{i}-\hat{y_{i}}\right)^2$$
Как видно, размерность MSE -- квадрат исходных данных. Иногда удобно извлечь корень, чтобы размерность совпадала с размерностью исходных данных. Такую метрику называют RMSE -- *среднеквадратическое отклонение*. "R" в начале расшифровывается как "root" (корень).

MSE квадратично штрафует за большие ошибки на объектах. Мы уже видели проявление этого при обучении моделей методом минимизации квадратичных ошибок – там это проявлялось в том, что модель старалась хорошо подстроиться под выбросы.

- $R^2$ ($R$-squared, $R$-квадрат, Коэффициент детерминации, Coefficient of determination) 
$$R^2 = 1 - \frac{\sum\limits_{i=1}^n\left(y_{i}-\hat{y_i} \right)^2}{\sum\limits_{i=1}^n\left(y_{i}-\bar{y_i} \right)^2}$$
Член вычитаемый из $1$ можно интерпретировать как оценённую дисперсию, отнесённую к реальной дисперсии. $R^2$  показывает, какая доля дисперсии таргетов (знаменатель) объяснена моделью. Ещё можно сказать, что мы модель сравниваем с моделью, которая предсказывает просто константу. У идеального решающего правила $R^2$ равен 1, у наилучшего константного предсказания он равен 0 на обучающей выборке.

- Mean Absolute Error (средняя абсолютная ошибка)
$$MAE = \frac{1}{n}\sum\limits_{i=1}^n\left|y_{i}-\hat{y_{i}}\right|$$
В случае работы с выборками с большим количеством выбросов прибегают к метрике MAE

Иногда бывает правильнее рассматривать не абсолютную, а относительную ошибку (например, когда в данных присутствуют разные по амплитуде значения).
- Mean Absolute Percentage Error (MAPE)
$$MAPE = \frac{1}{n}\sum\limits_{i=1}^n\left|\frac{y_{i}-\hat{y_{i}}}{y_{i}}\right|$$
В данном случае находим процент ошибки. Если в знаменателе 0, то либо на объекте выдают большой, но конечный штраф, либо пытаются перестаховаться и использовать *SMAPE* (symmetric mean absolute percentage error):

$$SMAPE = \frac{1}{n}\sum\limits_{i=1}^n\frac{2|y_{i}-\hat{y_{i}}|}{|y_{i}|+|\hat{y}_{i}|}$$
Когда есть большие выбросы в MAPE, следует использовать отнормированную и на прогнозируемое значение метрику.



### Метрики в задаче классификации
Будем рассматривать задачу бинарной классификации: необходимо по обучающей выборке ${(x_i, y_i)}^n_{i=1}$, где $y_i \in \{0, 1\}$, построить модель, которая по объекту $x$ предсказывает метку класса $f(x)\in\{0, 1\}$.
- Accuracy (доля правильных ответов)

$$Accuracy = \frac{1}{n}\sum\limits_{i=1}^n\left[y_i = \hat{y}_i \right].$$
Здесь использована нотация скобок Айверсона (Iverson bracket). Функция возвращает 1, если выражение внутри скобок истинно (True), и 0, если ложно (False).

У этой метрики есть недостатки. *Во-первых*, она не учитывает дисбаланс классов. Например, в задаче диагностики редких заболеваний классификатор, предсказывающий всем пациентам отсутствие болезни будет иметь достаточно высокую accuracy просто потому, что больных людей в выборке намного меньше. 

*Во-вторых*, она также не учитывает цену ошибки на объектах разных классов. Если ошибочный положительный диагноз для здорового больного обернётся лишь ещё одним обследованием, то ошибочно отрицательный вердикт может повлечь роковые последствия. 

Accuracy - это метрика, которая характеризует качество модели, агрегированное по всем классам. Это полезно, когда классы для нас имеют одинаковое значение. В случае, если это не так, accuracy может быть обманчивой.

Чтобы рассмотреть другие метрики, необходимо ввести понятие **матрицы ошибок**. Бинарная классификация может решаться в разных областях и зачастую это связано с обнаружением чего-либо. Например, поиск человека, больного туберкулёзом, по флюорографии, задача обнаружения сигнала в радиофизике и т.д. Поэтому класс, который представляет для нас интерес, называется "положительным", а оставшийся – "отрицательным".
Для каждого объекта в выборке возможны 4 ситуации:
1. **True positive (TP)** группа. мы предсказали положительный класс и угадали. True – потому что предсказали мы правильно, а positive – потому что предсказали положительную метку;
2. **False positive (FP)**. Мы предсказали положительную метку, но ошиблись в своём предсказании – (false, потому что предсказание было неправильным);
3. **True negative (TN)**. Предсказали отрицательную метку и угадали;
4. **False negative (FN)**. Предсказали отрицательную метку, но ошиблись. 

Для удобства все эти 4 числа изображают в виде таблицы, которую называют confusion matrix (матрицей ошибок):

<center><img src="data\\confusion_matrix.png" alt="drawing" width="700"/></center>

- Если мы рассмотрим долю правильно предсказанных положительных объектов среди всех объектов, предсказанных положительным классом, то мы получим метрику, которая называется **точностью (precision)**
$$ Precision = \frac{TP}{TP+FP}
$$

- Если же мы рассмотрим долю правильно найденных положительных объектов среди всех объектов положительного класса, то мы получим метрику, которая называется **полнотой (recall)**
$$ Recall = \frac{TP}{TP+FN}
$$
Например, в задаче предсказания злокачественности опухоли точность показывает, сколько из определённых нами как злокачественные опухолей действительно являются злокачественными, а полнота – какую долю злокачественных опухолей нам удалось выявить.

Стоит отметить, что точность и полнота не зависят от соотношения размеров классов. Даже если объектов положительного класса на порядки меньше, чем объектов отрицательного класса, данные показатели будут корректно отражать качество работы алгоритма.

- Удобно объединить Precision и Recall в одну метрику. Один из вариантов - взять их среднее гармоническое. Данный показатель эффективности исторически носит название **F1-меры (F1-measure)**
$$ F_1 = 2\frac{Recall\cdot Precision}{Recall + Precision}
$$
Если какая-то из метрик для нас приоритетнее, можно воспользоваться $F_\beta$ мерой:
$$ F_\beta = (\beta^2+1)\frac{Recall\cdot Precision}{Recall + \beta^2 Precision}.
$$


#### AUC-ROC
Многие модели бинарной классификации устроены так, что класс объекта получается бинаризацией выхода классификатора по некоторому фиксированному порогу. Например, модель логистической регрессии возвращает оценку вероятности принадлежности примера к положительному классу. Нам нужно учесть, что в зависимости от порога мы будем получать разные предсказания и разное качество на отложенной выборке. Так, чем ниже порог отсечения, тем больше объектов модель будет относить к положительному классу.

При уменьшении порога отсечения мы будем находить (правильно предсказывать) всё большее число положительных объектов, но также и неправильно предсказывать положительную метку на всё большем числе отрицательных объектов. Введем две метрики TPR и FPR:

$$ TPR = \frac{TP}{TP+FN}
$$
TPR (true positive rate) показывает долю верно принятых объектов;

$$ FPR = \frac{TP}{TP+FN}
$$
FPR (false positive rate) показывает долю неверно принятых объектов.

Кривая в осях TPR/FPR, которая получается при варьировании порога, исторически называется **ROC-кривой** (receiver operating characteristics curve, сокращённо ROC curve). Площадь под этой кривой (area under curve) может быть использована в качестве метрики, поэтому общее название данной метрики -- **AUC-ROC**